# pycisTarget

Import libraries necessaries

In [1]:
import pyranges as pr
from pycistarget.utils import region_names_to_coordinates
import scenicplus
import os
import pickle

Create the temporary and work directories

In [2]:
tmp_dir = 'output'
work_dir = 'output'
os.makedirs(work_dir, exist_ok=True)

### Open the regions found with pycisTopic

In [3]:
region_bin_topics_otsu = pickle.load(open(os.path.join('/home/roger/Github/OpenProblems/Conda/pycisTopic/output/scATAC/candidate_enhancers/region_bin_topics_otsu.pkl'), 'rb'))
region_bin_topics_top3k = pickle.load(open(os.path.join('/home/roger/Github/OpenProblems/Conda/pycisTopic/output/scATAC/candidate_enhancers/region_bin_topics_top3k.pkl'), 'rb'))
markers_dict = pickle.load(open(os.path.join('/home/roger/Github/OpenProblems/Conda/pycisTopic/output/scATAC/candidate_enhancers/markers_dict.pkl'), 'rb'))

### Create the region_set object

In [4]:
region_sets = {}
region_sets['topics_otsu'] = {}
region_sets['topics_top_3'] = {}
region_sets['DARs'] = {}
for topic in region_bin_topics_otsu.keys():
   regions = region_bin_topics_otsu[topic].index[region_bin_topics_otsu[topic].index.str.startswith('chr')] #only keep regions on known chromosomes
   region_sets['topics_otsu'][topic] = pr.PyRanges(region_names_to_coordinates(regions))
for topic in region_bin_topics_top3k.keys():
    regions = region_bin_topics_top3k[topic].index[region_bin_topics_top3k[topic].index.str.startswith('chr')] #only keep regions on known chromosomes
    region_sets['topics_top_3'][topic] = pr.PyRanges(region_names_to_coordinates(regions))
for DAR in markers_dict.keys():
    regions = markers_dict[DAR].index[markers_dict[DAR].index.str.startswith('chr')] #only keep regions on known chromosomes
    region_sets['DARs'][DAR] = pr.PyRanges(region_names_to_coordinates(regions))

In [5]:
for key in region_sets.keys():
    print(f'{key}: {region_sets[key].keys()}')

topics_otsu: dict_keys(['Topic1', 'Topic2', 'Topic3', 'Topic4', 'Topic5', 'Topic6', 'Topic7', 'Topic8', 'Topic9', 'Topic10', 'Topic11', 'Topic12', 'Topic13', 'Topic14', 'Topic15', 'Topic16', 'Topic17', 'Topic18', 'Topic19', 'Topic20', 'Topic21', 'Topic22', 'Topic23', 'Topic24', 'Topic25', 'Topic26', 'Topic27', 'Topic28', 'Topic29', 'Topic30'])
topics_top_3: dict_keys(['Topic1', 'Topic2', 'Topic3', 'Topic4', 'Topic5', 'Topic6', 'Topic7', 'Topic8', 'Topic9', 'Topic10', 'Topic11', 'Topic12', 'Topic13', 'Topic14', 'Topic15', 'Topic16', 'Topic17', 'Topic18', 'Topic19', 'Topic20', 'Topic21', 'Topic22', 'Topic23', 'Topic24', 'Topic25', 'Topic26', 'Topic27', 'Topic28', 'Topic29', 'Topic30'])
DARs: dict_keys(['B cells', 'Myeloid cells', 'NK cells', 'T cells CD4+', 'T cells CD8+', 'T regulatory cells'])


In [6]:
region_sets

{'topics_otsu': {'Topic1': +--------------+-----------+-----------+
  | Chromosome   | Start     | End       |
  | (category)   | (int64)   | (int64)   |
  |--------------+-----------+-----------|
  | chr1         | 221839548 | 221840443 |
  | chr1         | 159076610 | 159077485 |
  | chr1         | 27525394  | 27526227  |
  | chr1         | 15949316  | 15950233  |
  | ...          | ...       | ...       |
  | chrX         | 44898582  | 44899297  |
  | chrX         | 100931349 | 100932270 |
  | chrX         | 41331214  | 41332021  |
  | chrX         | 24168992  | 24169829  |
  +--------------+-----------+-----------+
  Unstranded PyRanges object has 4,550 rows and 3 columns from 23 chromosomes.
  For printing, the PyRanges was sorted on Chromosome.,
  'Topic2': +--------------+-----------+-----------+
  | Chromosome   | Start     | End       |
  | (category)   | (int64)   | (int64)   |
  |--------------+-----------+-----------|
  | chr1         | 24910830  | 24911730  |
  | chr1     

### Download rankings, scores and motif_annotation

You can download the data from the https://resources.aertslab.org/cistarget/

In [13]:
db_fpath = "~/Github/OpenProblems/Conda/pycisTopic/pycistarget/scenicplus"
motif_annot_fpath = "~/Github/OpenProblems/Conda/pycisTopic/pycistarget/scenicplus"

rankings_db = os.path.join(db_fpath, 'hg38_screen_v10_clust.regions_vs_motifs.rankings.feather')
scores_db =  os.path.join(db_fpath, 'hg38_screen_v10_clust.regions_vs_motifs.scores.feather')
motif_annotation = os.path.join(motif_annot_fpath, 'motifs-v10-nr.hgnc-m0.00001-o0.0.tbl')

Create the new directory called motifs

In [7]:
if not os.path.exists(os.path.join(work_dir, 'motifs')):
    os.makedirs(os.path.join(work_dir, 'motifs'))

### Create scenic+ object

In [11]:
import scenicplus
from scenicplus.wrappers.run_pycistarget import run_pycistarget
from scenicplus.wrappers.run_pycistarget import *
run_pycistarget(region_sets,
                 ctx_db_path = '/home/roger/Github/OpenProblems/Conda/pycisTopic/pycistarget/scenicplus/hg38_screen_v10_clust.regions_vs_motifs.rankings.feather',
                 dem_db_path = '/home/roger/Github/OpenProblems/Conda/pycisTopic/pycistarget/scenicplus/hg38_screen_v10_clust.regions_vs_motifs.scores.feather',
                 path_to_motif_annotations = '/home/roger/Github/OpenProblems/Conda/pycisTopic/pycistarget/scenicplus/motifs-v10nr_clust-nr.hgnc-m0.001-o0.0.tbl',
                
                 species = 'homo_sapiens',
                 save_path = '/home/roger/Github/OpenProblems/Conda/scenicplus/',
    
                 run_without_promoters = True,
                 biomart_host = 'http://www.ensembl.org',
                 promoter_space = 500,
                 ctx_auc_threshold = 0.005,
                 ctx_nes_threshold = 3.0,
                 ctx_rank_threshold = 0.05,
                 dem_log2fc_thr = 0.5,
                 dem_motif_hit_thr = 3.0,
                 dem_max_bg_regions = 500,
                
                 annotation_version = 'v10nr_clust',
                 annotation = ['Direct_annot', 'Orthology_annot'],
                 n_cpu = 1,
                 _temp_dir = '/home/roger/tmpdir_rayspill',)


2023-11-21 16:48:49,068 pycisTarget_wrapper INFO     /home/roger/Github/OpenProblems/Conda/scenicplus/ folder already exists.
2023-11-21 16:48:49,824 pycisTarget_wrapper INFO     Loading cisTarget database for topics_otsu
2023-11-21 16:48:49,826 cisTarget    INFO     Reading cisTarget database
2023-11-21 16:50:17,196 pycisTarget_wrapper INFO     Running cisTarget for topics_otsu
2023-11-21 16:50:17,198 cisTarget    INFO     Running cisTarget for Topic1 which has 7993 regions
2023-11-21 16:50:29,825 cisTarget    INFO     Annotating motifs for Topic1
2023-11-21 16:50:31,168 cisTarget    INFO     Getting cistromes for Topic1
2023-11-21 16:50:31,538 cisTarget    INFO     Running cisTarget for Topic2 which has 5978 regions
2023-11-21 16:50:41,207 cisTarget    INFO     Annotating motifs for Topic2
2023-11-21 16:50:42,530 cisTarget    INFO     Getting cistromes for Topic2
2023-11-21 16:50:42,727 cisTarget    INFO     Running cisTarget for Topic3 which has 4757 regions
2023-11-21 16:50:52,000 

Let's see if the data is OK?

In [ ]:
import pickle
with open('/home/roger/Github/OpenProblems/Conda/scenicplus/menr.pkl', 'rb') as f:
    data = pickle.load(f)

Let's see the results

In [ ]:
data['DEM_topics_top_3_All'].DEM_results('Topic1')